In [62]:
import pandas as pd

df_alimentos = pd.read_csv('/content/sample_data/alimentos.csv')


In [63]:
import random

def gerar_cardapio_aleatorio(df):
    alimentos = df['Descrição dos alimentos'].dropna().unique().tolist()

    cardapio = {
        'cafe': random.sample(alimentos, k=random.randint(2, 3)),
        'almoco': random.sample(alimentos, k=random.randint(2, 3)),
        'jantar': random.sample(alimentos, k=random.randint(2, 3))
    }

    return cardapio


In [64]:
def calcular_nutrientes_cardapio(cardapio, df):
    total = {
        'Energia..kcal.': 0,
        'Proteína..g.': 0,
        'Carboidrato..g.': 0,
        'Lipídeos..g.': 0
    }

    for refeicao, alimentos in cardapio.items():
        for alimento in alimentos:
            dados = df[df['Descrição dos alimentos'] == alimento]
            if not dados.empty:
                for chave in total.keys():
                    valor = dados[chave].values[0]
                    if pd.notna(valor):
                        total[chave] += valor

    return total


In [65]:
def calcular_fitness(nutrientes, metas):
    fitness = 0

    for chave in metas.keys():
        valor_real = nutrientes.get(chave, 0)
        valor_meta = metas[chave]
        desvio = abs(valor_real - valor_meta)

        fitness -= desvio

    return fitness

In [66]:
def selecao_por_torneio(populacao, fitnesses, k=3):
    selecionados = []

    for _ in range(len(populacao)):
        participantes = random.sample(list(zip(populacao, fitnesses)), k)
        vencedor = max(participantes, key=lambda x: x[1])
        selecionados.append(vencedor[0])

    return selecionados


In [67]:
def crossover(card1, card2):
    filho = {}

    for refeicao in ['cafe', 'almoco', 'jantar']:
        filho[refeicao] = random.choice([card1[refeicao], card2[refeicao]])

    return filho


In [68]:
def algoritmo_genetico(df, metas, geracoes=50, N=30):
    populacao = [gerar_cardapio_aleatorio(df) for _ in range(N)]

    for g in range(geracoes):
        fitnesses = [calcular_fitness(calcular_nutrientes_cardapio(c, df), metas) for c in populacao]

        nova_populacao = []
        selecionados = selecao_por_torneio(populacao, fitnesses)

        for i in range(0, N, 2):
            pai1 = selecionados[i]
            pai2 = selecionados[i + 1 if i + 1 < N else 0]
            filho1 = crossover(pai1, pai2)
            filho2 = crossover(pai2, pai1)
            nova_populacao.extend([mutacao(filho1, df), mutacao(filho2, df)])

        populacao = nova_populacao[:N]

    fitnesses = [calcular_fitness(calcular_nutrientes_cardapio(c, df), metas) for c in populacao]
    melhor_idx = fitnesses.index(max(fitnesses))

    return populacao[melhor_idx], fitnesses[melhor_idx]


In [69]:
def mutacao(cardapio, df, taxa=0.1):
    novo = cardapio.copy()

    for refeicao in novo:
        if random.random() < taxa:
            alimentos_possiveis = df['Descrição dos alimentos'].dropna().unique().tolist()
            if novo[refeicao]:
                idx = random.randint(0, len(novo[refeicao]) - 1)
                novo[refeicao][idx] = random.choice(alimentos_possiveis)

    return novo


In [70]:
metas = {
    'Energia..kcal.': 2000,
    'Proteína..g.': 75,
    'Carboidrato..g.': 275,
    'Lipídeos..g.': 70
}

melhor_cardapio, melhor_fitness = algoritmo_genetico(df_alimentos, metas)

print("\nMelhor cardápio:")
for r, itens in melhor_cardapio.items():
    print(f"{r}: {'; '.join(itens)}")

nutrientes = calcular_nutrientes_cardapio(melhor_cardapio, df_alimentos)
print("\nNutrientes:")
for n, v in nutrientes.items():
    print(f"{n}: {v:.2f}")

print(f"\nFitness final: {melhor_fitness:.2f}")



Melhor cardápio:
cafe: Pintado, cru; Abadejo, filé, congelado, assado
almoco: Sardinha, conserva em óleo; Frango, peito, sem pele, cozido; Macarrão, instantâneo
jantar: Biscoito, doce, wafer, recheado de morango; Mamão verde, doce em calda, drenado; Pinhão, cozido

Nutrientes:
Energia..kcal.: 1983.00
Proteína..g.: 106.10
Carboidrato..g.: 231.30
Lipídeos..g.: 74.10

Fitness final: -95.90
